# Introduction

Julia has a number of macros that can improve performance, sometimes considerably.

# Requirements

In [1]:
using BenchmarkTools

# Array views

Slicing an array in Julia creates a copy, as opposed to numpy.  Since this has an impact on performance, Julia defines a macro to indicate you want a view, not a copy.

To illustrate, consider a function to compute the sume of elements of an array.

In [2]:
function sum_vector(x)
    total = 0
    for i in 1:length(x)
        total += x[i]
    end
    return total
end

sum_vector (generic function with 1 method)

We can use this function to compute the column-wise sum of a two-dimensional array.

In [3]:
function sum_columns(A)
    total = zeros(size(A, 2))
    for i in 1:size(A, 2)
        total[i] = sum_vector(A[:, i])
    end
    return total
end

sum_columns (generic function with 1 method)

The second version of this function only differs in the application of the macro `@view`.

In [4]:
function sum_columns_view(A)
    total = zeros(size(A, 2))
    for i in 1:size(A, 2)
        total[i] = sum_vector(@view(A[:, i]))
    end
    return total
end

sum_columns_view (generic function with 1 method)

Let's verify the implementation.

In [5]:
A = [ 1 2 3; 4 5 6 ]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

In [6]:
sum_columns(A)

3-element Vector{Float64}:
 5.0
 7.0
 9.0

Now we can benchmark the two implementations.

In [7]:
A = rand(1000, 1000);

In [8]:
@benchmark sum_columns(A)

BechmarkTools.Trial: 1201 samples with 1 evaluations.
 Range (min … max):  3.502 ms …   6.916 ms  ┊ GC (min … max): 0.00% … 19.50%
 Time  (median):     3.771 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.121 ms ± 704.506 μs  ┊ GC (mean ± σ):  5.66% ±  9.43%

    ▄██▇▆▄▃▂ ▂    ▂           ▁▂ ▂▂▁▂                         ▁
  ▇███████████▇▇███▇▆▆▆▆▅▆▁▅▄▆████████▁█▆▇▇▇▇▇▆▆▆▅▇▇▇▄▇▇▆█▆▆▆ █
  3.5 ms       Histogram: log(frequency) by time      6.27 ms <

 Memory estimate: 7.76 MiB, allocs estimate: 1001.

In [9]:
@benchmark sum_columns_view(A)

BechmarkTools.Trial: 1932 samples with 1 evaluations.
 Range (min … max):  2.392 ms …   3.860 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.564 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.558 ms ± 109.650 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁         ▂█▂                                            
  ▂▃▂▃█▅▃▂▂▂▂▃▃▄████▅▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▂▁▂▂▂▂▂▂▂▁▂▁▁▂▂▂▁▂▁▂ ▃
  2.39 ms         Histogram: frequency by time         350 ms <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

The difference is quite significant, since using array views, we save on memory allocations and garbage collection.

# Array bounds checking

Although it is of course useful to have array bounds checked at runtime, this does generate some overhead.

Consider the following function that implements the vector triad daxpy.

In [10]:
function vector_triad(x, y, α)
    for i in 1:length(x)
        x[i] += α*x[i] + y[i]
    end
end

vector_triad (generic function with 1 method)

The second version of this function uses the `@inbounds` macro to indicate to the compiler that the programmer, i.e., you, is sure that any array access in the iteration will be within the bounds of the arrays.  Hence the compiler will not generate code to check.

In [11]:
function vector_triad_no_check(x, y, α)
    @inbounds for i in 1:length(x)
        x[i] += α*x[i] + y[i]
    end
end

vector_triad_no_check (generic function with 1 method)

Benchmarking this illustrates that, indeed, runtime array bounds checking generates significant overhead.

In [12]:
x = rand(100_000); y = rand(100_000); α = 3.1;

In [13]:
@benchmark vector_triad(x, y, α)

BechmarkTools.Trial: 10000 samples with 1 evaluations.
 Range (min … max):  81.701 μs …  1.673 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     82.102 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   86.846 μs ± 25.627 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▁▃ ▃      ▁                                                ▁
  ███▇███▇▇▇▇█▆▆▆▅▅▅▅▆▅▄▃▄▄▄▄▄▃▁▁▁▃▃▁▁▁▁▁▁▄▁▁▁▁▁▁▃▃▁▁▃▄▅▆▇▇██ █
  81.7 μs      Histogram: log(frequency) by time       204 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [14]:
@benchmark vector_triad_no_check(x, y, α)

BechmarkTools.Trial: 10000 samples with 1 evaluations.
 Range (min … max):  56.901 μs … 306.705 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     58.001 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   61.483 μs ±  17.726 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▂▁▁ ▃      ▁                                                ▁
  ██████▆█▆▇▇▆█▆▆▅▃▄▄▅▄▄▄▄▃▄▁▃▄▁▃▃▁▁▁▁▁▁▃▁▁▃▁▁▁▁▁▁▁▁▁▁▃▃▅▆▅▇▆▇ █
  56.9 μs       Histogram: log(frequency) by time       178 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

Using the `@inbounds` macro results in a reduced execution time.  However, it is important to realize that if you misjudge, and the array is accessed out of bounds, your code will not produce correct results and in the best case, it will crash.

# Single Instruction, Multiple Data (SIMD)

Julia has a macro to instruct the compiler to generate vector instructions.  In the implementation of the vector triad below, we add the `@simd` macro (note that the `@inbounds` macro is used as well.

In [15]:
function vector_triad_simd(x, y, α)
    @inbounds @simd for i in 1:length(x)
        x[i] += α*x[i] + y[i]
    end
end

vector_triad_simd (generic function with 1 method)

Benchmarking this version illustrates that the use of vector instruction can significantly improve performance.

In [16]:
x = rand(100_000); y = rand(100_000); α = 3.1;

In [17]:
@benchmark vector_triad_no_check(x, y, α)

BechmarkTools.Trial: 10000 samples with 1 evaluations.
 Range (min … max):  57.601 μs …  1.668 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     58.301 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   61.959 μs ± 29.384 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▂▁ ▃▁ ▁                                                    ▁
  ██████▆█▆▆▆██▅▅▅▄▅▅▄▃▄▄▄▃▃▁▁▄▃▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▃▄▆▆▅▆▇ █
  57.6 μs      Histogram: log(frequency) by time       178 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [18]:
@benchmark vector_triad_simd(x, y, α)

BechmarkTools.Trial: 10000 samples with 1 evaluations.
 Range (min … max):  25.100 μs … 203.703 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     25.700 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   27.522 μs ±  12.962 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▂▁▃ ▁                                                       ▁
  ██████▇▆▄▄▅▄▁▁▃▁▁▃▃▁▁▁▁▁▃▁▃▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▃ █
  25.1 μs       Histogram: log(frequency) by time       129 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.